In [ ]:
import json
import os

def process_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    metrics = data['metrics']
    
    # Process misclassifications
    metrics['misclassifications'] = [
        m for m in metrics['misclassifications']
        if m['gpt_classification'] != m['ground_truth_classification']
    ]
    
    # Update correct classifications and accuracy
    metrics['correct_classifications'] = metrics['total_substeps'] - len(metrics['misclassifications'])
    metrics['accuracy'] = metrics['correct_classifications'] / metrics['total_substeps']
    
    # Process confusion matrix
    confusion_matrix = metrics['confusion_matrix']
    new_matrix = {k: {} for k in ['VA', 'BVA', 'NVA']}
    
    for true_class, predictions in confusion_matrix.items():
        if true_class == 'Unclassified':
            for pred_class, count in predictions.items():
                if pred_class in new_matrix:
                    new_matrix[pred_class][pred_class] = new_matrix[pred_class].get(pred_class, 0) + count
        else:
            for pred_class, count in predictions.items():
                if pred_class == 'Unclassified':
                    new_matrix[true_class][true_class] = new_matrix[true_class].get(true_class, 0) + count
                else:
                    new_matrix[true_class][pred_class] = new_matrix[true_class].get(pred_class, 0) + count
    
    metrics['confusion_matrix'] = new_matrix
    
    return data

def process_directory(directory_path):
    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)
            processed_data = process_json_file(file_path)
            
            # Save the processed data
            output_path = os.path.join(directory_path, f'processed_{filename}')
            with open(output_path, 'w') as file:
                json.dump(processed_data, file, indent=4)
            
            print(f"Processed {filename} and saved as processed_{filename}")

def process_directory(base_dir):
    records = []
    for sector in os.listdir(base_dir):
        sector_path = os.path.join(base_dir, sector)
        if os.path.isdir(sector_path):
            for activity in os.listdir(sector_path):
                activity_path = os.path.join(sector_path, activity)
                if os.path.isdir(activity_path):
                    for filename in os.listdir(activity_path):
                        if filename.endswith("_response.json") and "Value-Classification" in filename:


# Set the base directory
ROOT_DIR = r"C:\Projects\Research\SWEEP\SWEEP"
base_directory = r"test\results\value_adding_analysis"
data_df = process_directory(os.path.join(ROOT_DIR, base_directory))